In [1]:
"""
in the end i want to have a file with all the values of the actors - i dont want to

what i can do is, i take a value, check if the value exists in the dataset and if yes 
"""
# https://api.github.com/users/Cydrobolt
# https://api.github.com/users/jspc
# https://api.github.com/users/vt100
# https://api.github.com/users/viesturz
# https://api.github.com/users/4pr0p02
# https://api.github.com/users/nerr
# https://api.github.com/users/haithemChkel
# https://api.github.com/users/rsisx9
# https://api.github.com/users/mpa1212
import requests

# List of GitHub user endpoints
user_urls = [
    "https://api.github.com/users/Cydrobolt",
    "https://api.github.com/users/jspc",
    "https://api.github.com/users/vt100",
    "https://api.github.com/users/viesturz",
    "https://api.github.com/users/4pr0p02",
    "https://api.github.com/users/nerr",
    "https://api.github.com/users/haithemChkel",
    "https://api.github.com/users/rsisx9",
    "https://api.github.com/users/mpa1212"
]

# Loop through each URL and make a GET request
for url in user_urls:
    response = requests.get(url)
    if response.status_code == 200:
        print(f"Data for {url}: {response.json()}")
    else:
        print(f"Failed to retrieve data from {url}, status code: {response.status_code}")

Data for https://api.github.com/users/Cydrobolt: {'login': 'cydrobolt', 'id': 1058150, 'node_id': 'MDQ6VXNlcjEwNTgxNTA=', 'avatar_url': 'https://avatars.githubusercontent.com/u/1058150?v=4', 'gravatar_id': '', 'url': 'https://api.github.com/users/cydrobolt', 'html_url': 'https://github.com/cydrobolt', 'followers_url': 'https://api.github.com/users/cydrobolt/followers', 'following_url': 'https://api.github.com/users/cydrobolt/following{/other_user}', 'gists_url': 'https://api.github.com/users/cydrobolt/gists{/gist_id}', 'starred_url': 'https://api.github.com/users/cydrobolt/starred{/owner}{/repo}', 'subscriptions_url': 'https://api.github.com/users/cydrobolt/subscriptions', 'organizations_url': 'https://api.github.com/users/cydrobolt/orgs', 'repos_url': 'https://api.github.com/users/cydrobolt/repos', 'events_url': 'https://api.github.com/users/cydrobolt/events{/privacy}', 'received_events_url': 'https://api.github.com/users/cydrobolt/received_events', 'type': 'User', 'site_admin': False

In [3]:
import requests

# Path to the file containing the URLst
file_path = 'github/2015-04-actor-urls.txt'

# Open the file and read each line
with open(file_path, 'r') as file:
    for line in file:
        url = line.strip()  # Remove any leading/trailing whitespace
        response = requests.get(url)
        if response.status_code == 200:
            print(f"Data for {url}: {response.json()}")
        else:
            print(f"Failed to retrieve data from {url}, status code: {response.status_code}")

Data for https://api.github.com/users/Cydrobolt: {'login': 'cydrobolt', 'id': 1058150, 'node_id': 'MDQ6VXNlcjEwNTgxNTA=', 'avatar_url': 'https://avatars.githubusercontent.com/u/1058150?v=4', 'gravatar_id': '', 'url': 'https://api.github.com/users/cydrobolt', 'html_url': 'https://github.com/cydrobolt', 'followers_url': 'https://api.github.com/users/cydrobolt/followers', 'following_url': 'https://api.github.com/users/cydrobolt/following{/other_user}', 'gists_url': 'https://api.github.com/users/cydrobolt/gists{/gist_id}', 'starred_url': 'https://api.github.com/users/cydrobolt/starred{/owner}{/repo}', 'subscriptions_url': 'https://api.github.com/users/cydrobolt/subscriptions', 'organizations_url': 'https://api.github.com/users/cydrobolt/orgs', 'repos_url': 'https://api.github.com/users/cydrobolt/repos', 'events_url': 'https://api.github.com/users/cydrobolt/events{/privacy}', 'received_events_url': 'https://api.github.com/users/cydrobolt/received_events', 'type': 'User', 'site_admin': False

KeyboardInterrupt: 

In [18]:
import requests
import concurrent.futures
import json

# Function to load proxies from file
def load_proxies(filepath):
    proxies = []
    with open(filepath, 'r') as file:
        for line in file:
            parts = line.strip().split(':')
            if len(parts) == 4:  # Ensure there are four parts: host, port, user, password
                host, port, user, password = parts
                formatted_proxy = f"http://{user}:{password}@{host}:{port}"
                proxies.append({
                    'http': formatted_proxy,
                    'https': formatted_proxy
                })
    return proxies

# Function to load URLs from file, limited to the first 1000 URLs
def load_urls(filepath):
    urls = []
    with open(filepath, 'r') as file:
        for line in file:
            if len(urls) < 1000:  # Limit to first 1000 URLs
                urls.append(line.strip())
            else:
                break
    return urls

# Function to make a request using a specific proxy
def fetch_url(url, proxy):
    try:
        response = requests.get(url, proxies=proxy)
        if response.status_code == 200:
            return response.json()  # Return only the JSON data from the response
        else:
            return {'error': f"Failed to retrieve data, status code: {response.status_code}"}
    except requests.exceptions.RequestException as e:
        return {'error': str(e)}

# Load proxies and URLs
proxies = load_proxies('ips-data_center.txt')
user_urls = load_urls('github/2015-01-actor-urls.txt')

if not proxies:
    print("Error: No proxies loaded. Check the proxy file and format.")
    exit()

# Function to get a proxy from the list, rotating them
def get_proxy(index):
    return proxies[index % len(proxies)]

# Use ThreadPoolExecutor to make requests in parallel and write results to a JSON file
with concurrent.futures.ThreadPoolExecutor(max_workers=1000) as executor, open('results.json', 'w') as results_file:
    results = []
    future_to_url = {executor.submit(fetch_url, url, get_proxy(index)): url for index, url in enumerate(user_urls)}
    for future in concurrent.futures.as_completed(future_to_url):
        try:
            result = future.result()
            results.append(result)
        except Exception as exc:
            print(f"An exception occurred: {exc}")
    json.dump(results, results_file, indent=4)  # Serialize and save the results to a JSON file

In [ ]:
"""
workers, seconds
50, 25.1
200, 7.6
500, 6.6
1000, 8.2

1. plan for getting the final dataset - remove all duplicates from all combined files
2. iterate over all files and each line of the files and write the request response to a json file
3. deal with cases of errors because of rate limits, error because the profile doesnt exist anymore and 
"""

